In [54]:
import pandas as pd
import os
cwd = os.getcwd() # change for '__file__' if using .py 

filename = (cwd + '\\mvps.csv')
mvps = pd.read_csv(filename)
mvps = mvps[['Player','Age','Pts Won','Pts Max','Share','Year']]

filename = (cwd + '\\player_stats.csv')
players = pd.read_csv(filename)
players['Player'] = players['Player'].str.replace('*','',regex=False)                   # remove the '*' so the names in "playerss" match those in "mvps"

def single_row(df):                                                                     # check if the player data frame has one or more entries (i.e if they played in more than one team)
    if df.shape[0] ==1:
        return df
    else:
        row = df[df['Tm'] == 'TOT']                                                     # use the line of data for their total
        row['Tm'] = df.iloc[-1,:]['Tm']                                                 # replacae the name "TOT" for the last entry (i.e. "iloc[-1,:]"")
        return row

players = players.groupby(['Player','Year']).apply(single_row)                          # in this case the df "player" has two index levels / groups (similar to structs in matlab)
# players.head(20) 

players.index = players.index.droplevel()                                               # remove the two levels from the data 
players.index = players.index.droplevel()
del players['Unnamed: 0']
del players['Rk']

In [68]:
all_players = players.merge(mvps, how='outer',on=['Player','Year']) # outter = if you cant find the data in mvps, keep the data anyways ("inner" would keep only the common between players and mvps)
# combined[combined['Pts Won']>0]  # check only players with more than 0 points won
all_players[['Pts Won','Pts Max','Share']] = all_players[['Pts Won','Pts Max','Share']].fillna(0)
all_players.head(3)

,Player,Pos,Age_x,Tm,G,GS,MP,FG,FGA,FG%,...,STL,BLK,TOV,PF,PTS,Year,Age_y,Pts Won,Pts Max,Share
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,.476,...,0.7,0.3,1.2,1.4,9.1,1991,NaN,0.0,0.0,0.0
1,A.C. Green,PF,28,LAL,82,53,35.4,4.7,9.8,.476,...,1.1,0.4,1.4,1.7,13.6,1992,NaN,0.0,0.0,0.0
2,A.C. Green,PF,29,LAL,82,55,34.4,4.6,8.6,.537,...,1.1,0.5,1.4,1.8,12.8,1993,NaN,0.0,0.0,0.0


In [73]:
teams = pd.read_csv((cwd + '\\team_stats.csv'))
teams = teams[~teams['W'].str.contains('Division')]
teams['Team']   = teams['Team'].str.replace('*','',regex=False)                            # remove the '*' so the names in "playerss" match those in "mvps"

teams.head(5)

nicknames = {}                                                                           # create a map with the names of teams and their abbreviation
with open((cwd + '\\nicknames.csv')) as f: 
    lines = f.readlines()
    for l in lines[1:]:                                                                 # loop by skiping the nth row = [n:]
        abbrev   ,name = l.replace('\n','').split(',')
        nicknames[abbrev] = name

all_players['Team'] = all_players['Tm'].map(nicknames)
all_players.head(5)

,Player,Pos,Age_x,Tm,G,GS,MP,FG,FGA,FG%,...,BLK,TOV,PF,PTS,Year,Age_y,Pts Won,Pts Max,Share,Team
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,.476,...,0.3,1.2,1.4,9.1,1991,NaN,0.0,0.0,0.0,Los Angeles Lakers
1,A.C. Green,PF,28,LAL,82,53,35.4,4.7,9.8,.476,...,0.4,1.4,1.7,13.6,1992,NaN,0.0,0.0,0.0,Los Angeles Lakers
2,A.C. Green,PF,29,LAL,82,55,34.4,4.6,8.6,.537,...,0.5,1.4,1.8,12.8,1993,NaN,0.0,0.0,0.0,Los Angeles Lakers
3,A.C. Green,PF,30,PHO,82,55,34.5,5.7,11.3,.502,...,0.5,1.2,1.7,14.7,1994,NaN,0.0,0.0,0.0,Phoenix Suns
4,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,.504,...,0.4,1.4,1.8,11.2,1995,NaN,0.0,0.0,0.0,Phoenix Suns


In [ ]:
# df.merge(df_to_add,)
stats = all_players.merge(teams,how='outer',on=['Team','Year'])  # for every row in combined assigne the corresponding team
del stats['Unnamed: 0']

if len(stats) != len(all_players):                                                 # check if df are same size. They should be
      print('\x1b[7;31;40m' + 'data frames are not the same length' + '\x1b[0m')   # check more colours in https://stackoverflow.com/questions/287871/how-do-i-print-colored-text-to-the-terminal

stats = stats.apply(pd.to_numeric,errors='ignore')                                 # convert all colimns to numeric

stats['GB'].unique()                                                               # here you can see that 'GB' (games back) in stats is not numeric because of a '-' in one of the cells
stats['GB'] = stats['GB'].str.replace('—','0')
stats['GB'].unique()  
stats = stats.apply(pd.to_numeric,errors='ignore')

stats.dtypes
stats.to_csv((cwd + '\player_mvp_stats.csv'))

In [ ]:
import pandas as pd
import os

cwd = os.getcwd() # change for '__file__' if using .py 

stats = pd.read_csv((cwd + '\player_mvp_stats.csv'))  
highest_scoring = stats[stats['G']>70].sort_values('PTS', ascending=False).head(8)
highest_scoring.plot.bar('Player','PTS')
highest_scoring.plot.bar('Player','3P%')

In [ ]:
heighest_score_perYear = stats.groupby('Year').apply(lambda x: x.sort_values('PTS',ascending=False).head(1))
ax = heighest_score_perYear.plot.bar('Year','PTS')
ax.set_xticklabels(heighest_score_perYear['Player'])
ax.set_xlabel('1991 to 2021')

In [ ]:
heighest_3PT_percentage = stats[stats['3PA']>1].groupby('Year').apply(lambda x: x.sort_values('3P%',ascending=False).head(1))
ax = heighest_3PT_percentage.plot.bar('Year','3P%')
ax.set_xticklabels(heighest_3PT_percentage['Player'])
ax.set_xlabel('1991 to 2021')
ax.legend(('more that 2x3PA'))

In [ ]:
corr = stats.corr()[['Share','3P%']]
corr[abs(corr)>0.5]